# Modelling the OLG Model

# Introduction

We have chosen to analyze an Overlapping Generations model (**OLG**) in a Pay-As-You-Go (**PAYG**) and Fully Funded system. The OLG-model is widely used to study macroeconomic dynamics and economic growth in an economy.

In contrast to e.g., a Ramsey-Cass-Koopman growth model where individuals are infinitely lived, an OLG-model assumes the agents in the economy to be finitely lived. That is, agents live for two "periods", first period when they are young and second period, when they are old. This makes the model a bit more realistic from our point of view, and therefore also more interesting to analyze.

We employ the OLG-model to find the capital per. capita accumulation and capital per. capita in steady state, as well as the agents' intertemporal substitution elasticity expressed throught the Euler-equation.

We will first look at the difference in the capital per. capita accumulation and capital per. capita in steady state between a PAYG and FF system in a simple OLG model. Further we will then make an extension to the simple OLG model, so it includes population growth, to see how the capital per. capita accumulation and capital per. capita in steady state for the PAYG and FF system differentiate from the simple OLG model. Finally, we will look at how the values of the parameters influence the accumulation of capital per. capita and capital per. capita in steady state for the PAYG and FF system.

Imports and set magics:

In [310]:
from types import SimpleNamespace
from scipy import optimize
import numpy as np
import sympy as sm
import matplotlib.pyplot as plt
import ipywidgets as widgets
from OLGmodel import OLG_Class


# autoreload modules when code is run
%load_ext autoreload
%autoreload 2



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Model description

**Firms**: In our OLG model the competitive firms are identical and maximize their profits:

$$
\begin{aligned}
\pi_t & =\max_{K_{t},L_t} AK_t^\alpha L_t^{1-\alpha}-r_tK_t-w_tL_t\\
\end{aligned}
$$
, where $K_{t}$ is the capital used in production, $L_t$ is the labor employed in the production and $A$ is the technology level (constant). We use a Cobb-Douglas production-function, where $\alpha$ represents the share of capital used in production, $\alpha \in (0, 1)$.

We use, that 
$$
\begin{aligned}
\text{Capital per capita}: k_t & =\frac{K_t}{L_t} \\
\text{Rent}:  r_t & = \alpha A k_t^{\alpha-1} \\
\text{Wage}: w_t & = (1-\alpha) A k_t^{\alpha}
\end{aligned}
$$

The wage and rent are equal to their respective marginal product 

**Demographics:** In our OLG model life consists of two periods, $t$ and $t+1$. The households can either be young or old. When they are young they work, and when they are old they do not work. The young households therefore save some share of their wage for them to have when they are old. The population is also constant, so $L_t=L_{t+1}$.

**Households:** Utility for the young households is:
    
$$
\begin{aligned}
U_t & = \log(c_{1t})+\frac{1}{1+\rho} \log(c_{2t+1})\\
\end{aligned}
$$
, where $\rho$ represents the patience of young households between consumption in the first period (when young) and second period (when old). The higher $\rho$ is the more impatient the young households is, $\rho \in (0,1)$.

**Pension systems:** We implement two different pension systems, PAYG and FF, which affect the budget constraints.

In the PAYG system the young households must contribute a share of their wage/income, $\tau \in (0,1)$, to the old households, in the same period as them. 
The following budget constraints is for the Pay As You Go (PAYG) system:
$$
\begin{aligned}
c_{1t} & = (1-\tau)w_t-s_t
\\
c_{2t+1} & = (1+r_{t+1})s_t+w_{t+1}\tau
\end{aligned}
$$
, where $c_{1t}$ is consumption in the first period (when young) and $c_{2t+1}$ is consumption in the second period (when old). $s_t$ is the savings the young households make, which they will recieve when they are old in the next period. They invest their savings in capital, so they get an interest, $r_{t+1}$, on their savings. 

In the FF system the young households must contribute a share of their wage/income, $\tau \in (0,1)$, but now it is for themselves when they are old in the next period. 
The following budget constraints is for a Fully Funded (FF) system:
$$
\begin{aligned}
c_{1t} & = (1-\tau)w_t-s_t
\\
c_{2t+1} & = (1+r_{t+1})(s_t+w_{t+1}\tau)
\end{aligned}
$$
, because the contribution from the young households functions as the saving, they invest the their contribution in capital and receive an interest in the next period, when they are old, just like their savings. We assume the interest for the saving and contribution to be the same.


**Capital:** We assume that the old spend their entrie savings, so $\delta = 1 $.

**Steady state:**

1. Find Euler equation
2. Find $s_t$ in the Euler equation by inserting the budget constraints, then $r_{t}, r_{t+1}, w_t, w_{t+1}$ and at last isolate $s_t$
3. Find capital per. capita accumulation: $k_{t+1}=(1-\delta)k_t+s_t$ in PAYG and $k_{t+1}=(1-\delta)k_t+s_t+\tau w_t$ in FF.
4. In steady state $k_{t+1}=k_{t}=k^{*}$

## Analytical solutions for PAYG and FF system

In [311]:
#define symbols 
alpha = sm.symbols('alpha')
k_t = sm.symbols('k_t')
k_t1 = sm.symbols('k_{t+1}')
k_ss = sm.symbols('k^*')
A = sm.symbols('A')
w_t = sm.symbols('w_t')
w_t1 = sm.symbols('w_{t+1}') 
r_t = sm.symbols('r_{t+1}')
r_t1 = sm.symbols('r_{t+1}')
s_t = sm.symbols('s_t') 
tau = sm.symbols('tau')
c_1t = sm.symbols('c_1t') 
c_2t = sm.symbols('c_{2t+1}') 
rho = sm.symbols('rho') 
lambdaa = sm.symbols('lambda')
U_t = sm.symbols('U_t')

#define rent and wage in period t and t+1
rent_t = alpha*A*k_t**(alpha-1)
rent_t1 = alpha*A*k_t1**(alpha-1)
wage_t = (1-alpha)*A*k_t**alpha
wage_t1 = (1-alpha)*A*k_t1**alpha

**Euler equation:**

In [312]:
#define utility function and budget constraints for PAYG and FF systems
u = sm.log(c_1t) + 1/(1+rho)*sm.log(c_2t)
con1 = sm.Eq(c_1t+s_t, (1-tau)*w_t)
con2_payg = sm.Eq(c_2t, (1+r_t1)*s_t+tau*w_t1)
con2_FF = sm.Eq(c_2t, (1+r_t1)*s_t+(1+r_t1)*tau*w_t)

#isolate s_t in con2 and substitute it into con1, so we end up with one budget constraint for both systems
st_con2_PAYG = sm.solve(con2_payg, s_t)
st_con2_FF = sm.solve(con2_FF, s_t)
con1_sub_PAYG = con1.subs(s_t, st_con2_PAYG[0])
con1_sub_FF = con1.subs(s_t, st_con2_FF[0])

# Move right side of budget constraint to the left side for both systems
con_final_PAYG = sm.solve(con1_sub_PAYG, (1-tau)*w_t)[0] - (1-tau)*w_t
con_final_FF = sm.solve(con1_sub_FF, (1-tau)*w_t)[0] - (1-tau)*w_t


#Define lagrangian function with u and con_final for both systems
lagrangian_func_PAYG = u - lambdaa*con_final_PAYG
lagrangian_func_FF = u - lambdaa*con_final_FF

#Take the derivative wrt. c_1t and c_2t for both lagrangian function
foc1_PAYG = sm.diff(lagrangian_func_PAYG, c_1t)
foc2_PAYG = sm.diff(lagrangian_func_PAYG, c_2t)
foc1_FF = sm.diff(lagrangian_func_FF, c_1t)
foc2_FF = sm.diff(lagrangian_func_FF, c_2t)

#isolate lambda in the derivatives
iso_foc1_PAYG = sm.solve(foc1_PAYG,lambdaa)[0]
iso_foc2_PAYG = sm.solve(foc2_PAYG,lambdaa)[0]
iso_foc1_FF = sm.solve(foc1_FF,lambdaa)[0]
iso_foc2_FF = sm.solve(foc2_FF,lambdaa)[0]

#set the two isolate lamdaa equal to each other for both systems, so we get two Euler equation
euler_PAYG = sm.Eq(iso_foc1_PAYG,iso_foc2_PAYG)
final_euler_PAYG = sm.Eq(c_1t,sm.solve(euler_PAYG,c_1t)[0])
euler_FF = sm.Eq(iso_foc1_FF,iso_foc2_FF)
final_euler_FF = sm.Eq(c_1t,sm.solve(euler_FF,c_1t)[0])

print('Euler equation in the PAYG system:')
display(final_euler_PAYG)
print('')
print('Euler equation in the FF system:')
display(final_euler_FF)

Euler equation in the PAYG system:


Eq(c_1t, c_{2t+1}*(rho + 1)/(r_{t+1} + 1))


Euler equation in the FF system:


Eq(c_1t, c_{2t+1}*(rho + 1)/(r_{t+1} + 1))

We see that the Euler equation is the same for both the PAYG and FF system

**Savings $(s_t)$**: From Euler equation


In [313]:
#isolate c_1t and c_2t in budget constraints for both systems and substitute it into Euler equation 
iso_con1 = sm.solve(con1,c_1t)[0]
iso_con2_payg = sm.solve(con2_payg,c_2t)[0]
iso_con2_FF = sm.solve(con2_FF,c_2t)[0]
savings_PAYG = final_euler_PAYG.subs(c_1t,iso_con1).subs(c_2t, iso_con2_payg)
savings_FF = final_euler_FF.subs(c_1t,iso_con1).subs(c_2t, iso_con2_FF)

#isolate s_t in savings for both systems
final_savings_PAYG = sm.Eq(s_t, sm.solve(savings_PAYG,s_t)[0])
final_savings_FF = sm.Eq(s_t, sm.solve(savings_FF,s_t)[0])
print('Savings in the PAYG system:')
display(final_savings_PAYG)
print('')
print('Savings in the FF system:')
display(final_savings_FF)

Savings in the PAYG system:


Eq(s_t, (-r_{t+1}*tau*w_t + r_{t+1}*w_t - rho*tau*w_{t+1} - tau*w_t - tau*w_{t+1} + w_t)/(r_{t+1}*rho + 2*r_{t+1} + rho + 2))


Savings in the FF system:


Eq(s_t, w_t*(-rho*tau - 2*tau + 1)/(rho + 2))

We would liked to have a more simplified equation for $s_t$ in the PAYG system, as for example: 
$$
\begin{aligned}
s_t = \frac{1-\rho}{2+\rho}w_t - \tau \frac{1+\rho}{(2+\rho)(1+r_{t+1})}w_{t+1}
\end{aligned}
$$
, but Sympy/Python cannot do that for us. They are still the same, one is just more simplified.

We would again like to have a more simplified equation for $s_t$ in the FF system, as for example:
$$
\begin{aligned}
s_t = (\frac{1}{2+\rho}-\tau)w_t
\end{aligned}
$$
, but Sympy/Python cannot do that for us. They are still the same, one is just more simplified. 

**Capital accumulation**: from savings as $k_{t+1}=s_t$ in PAYG and $k_{t+1}=s_t+w_t\tau$ in FF

In [314]:
#substitute s_t with k_t1 in both savings equations
cap_acc_PAYG = sm.Eq(k_t1, sm.solve(savings_PAYG,s_t)[0])
cap_acc_FF = sm.Eq(k_t1, sm.solve(savings_FF,s_t)[0] + tau*w_t)

#substitute w_t, w_t1 and r_t1 with wage_t, wage_t1 and rent_t1 in both of the above equations
sub_cap_acc_PAYG = (((cap_acc_PAYG.subs(w_t,wage_t)).subs(w_t1,wage_t1)).subs(r_t1,rent_t1)).subs(s_t,k_t1)
final_cap_acc_FF = cap_acc_FF.subs(w_t,wage_t)

#isolate k_t1 in sub_cap_acc_PAYG, but sympy can't do it since it did not simplifed final_savings (too complex)
"final_cap_acc_PAYG = sm.solve(sub_cap_acc_PAYG,k_t1)"

#Instead we show
print('almost capital accumulation in the PAYG system:')
display(sub_cap_acc_PAYG)



almost capital accumulation in the PAYG system:


Eq(k_{t+1}, (-A**2*alpha*k_t**alpha*k_{t+1}**(alpha - 1)*tau*(1 - alpha) + A**2*alpha*k_t**alpha*k_{t+1}**(alpha - 1)*(1 - alpha) - A*k_t**alpha*tau*(1 - alpha) + A*k_t**alpha*(1 - alpha) - A*k_{t+1}**alpha*rho*tau*(1 - alpha) - A*k_{t+1}**alpha*tau*(1 - alpha))/(A*alpha*k_{t+1}**(alpha - 1)*rho + 2*A*alpha*k_{t+1}**(alpha - 1) + rho + 2))

If sympy could have isolated $k_t1$ in sub_cap_acc, then we would have gotten a very complex edition of 
$$
\begin{aligned}
k_{t+1} = \frac{(1-\alpha)(1-\tau)\alpha}{(2+\rho)\alpha+(1+\rho)(1-\alpha)\tau}A k_t^\alpha 
\end{aligned}
$$
, but they would still be the same.

We will go by the simplified capital accumulation above for the rest of the project

In [315]:
final_cap_acc_PAYG = sm.Eq(k_t1, (1-alpha)*(1-tau)*alpha/((2+rho)*alpha+(1+rho)*(1-alpha)*tau)*A*k_t**alpha)
print('Capital accumulation in the PAYG system:')
display(final_cap_acc_PAYG)
print('')
print('Capital accumulation in the FF system:')
display(final_cap_acc_FF)

Capital accumulation in the PAYG system:


Eq(k_{t+1}, A*alpha*k_t**alpha*(1 - alpha)*(1 - tau)/(alpha*(rho + 2) + tau*(1 - alpha)*(rho + 1)))


Capital accumulation in the FF system:


Eq(k_{t+1}, A*k_t**alpha*tau*(1 - alpha) + A*k_t**alpha*(1 - alpha)*(-rho*tau - 2*tau + 1)/(rho + 2))

**Steady State**: From capital accumulation as $k_{t+1}=k_t=k^*$

In [316]:
#substitute k_t1 and k_t with k_ss in both of the capial accumulations
ss_PAYG = (final_cap_acc_PAYG.subs(k_t1,k_ss)).subs(k_t,k_ss)
ss_FF = (final_cap_acc_FF.subs(k_t1,k_ss)).subs(k_t,k_ss)

#isolate k_ss in both of the capial accumulations after the substitution
final_ss_PAYG = sm.Eq(k_ss, sm.solve(ss_PAYG,k_ss)[0])
'final_ss_FF = sm.Eq(k_ss, sm.solve(ss_FF,k_ss)[0])' #No algorithms are implemented to solve the equation, so we do it by hand
final_ss_FF = sm.Eq(k_ss, (((-rho*tau - 2*tau + 1)/(rho + 2)+tau)*(1-alpha)*A)**(1/(1-alpha)))
print('Steady state in the PAYG system:')
display(final_ss_PAYG)
print('')
print('Steady state in the FF system:')
display(final_ss_FF)

Steady state in the PAYG system:


Eq(k^*, ((-alpha*rho*tau + alpha*rho - alpha*tau + 2*alpha + rho*tau + tau)/(A*alpha*(alpha*tau - alpha - tau + 1)))**(1/(alpha - 1)))


Steady state in the FF system:


Eq(k^*, (A*(1 - alpha)*(tau + (-rho*tau - 2*tau + 1)/(rho + 2)))**(1/(1 - alpha)))

## Numerical solution

In [317]:
model = OLG_Class()
par = model.par # SimpeNamespace
sim = model.sim # SimpeNamespace
k_ss = 2
par.k_lag_ini = 0.1*k_ss

initializing the model:
calling .setup()
calling .allocate()


In [318]:
from OLGmodel import simulate_before_s, simulate_after_s, find_s_bracket, calc_euler_error

In [319]:
s_guess = 0.41

In [320]:
# a. initialize
sim.k_lag[0] = par.k_lag_ini

simulate_before_s(par,sim,t=0,s=s_guess)
print(f'{sim.C2[0] = : .4f}')

simulate_after_s(par,sim,s=s_guess,t=0)
print(f'{sim.C1[0] = : .4f}')

simulate_before_s(par,sim,t=1,s=s_guess)
print(f'{sim.C2[1] = : .4f}')
print(f'{sim.r[1] = : .4f}')

LHS_Euler = sim.C1[0]
RHS_Euler = (1+par.rho)*sim.C2[1]/(sim.r[1]+1)
print(f'euler-error = {LHS_Euler-RHS_Euler:.8f}')

sim.C2[0] =  0.8759
sim.C1[0] = -0.0645
sim.C2[1] =  0.6656
sim.r[1] =  0.2611
euler-error = -0.72424435


In [321]:
euler_error = calc_euler_error(s_guess,par,sim,t=0)
print(f'euler-error = {euler_error:.8f}')

euler-error = -0.72424435


In [322]:
s_min,s_max = find_s_bracket(par,sim,t=0,do_print=True);

euler-error for s =   0.99999999 =  -2.05174433
euler-error for s =   0.50000000 =  -0.92674435
euler-error for s =   0.25000001 =  -0.36424436
euler-error for s =   0.12500001 =  -0.08299437
euler-error for s =   0.06250001 =   0.05763063
bracket to search in with opposite signed errors:
[  0.06250001-  0.12500001]


In [323]:
obj = lambda s: calc_euler_error(s,par,sim,t=0)
result = optimize.root_scalar(obj,bracket=(s_min,s_max),method='bisect')
print(result)

      converged: True
           flag: 'converged'
 function_calls: 37
     iterations: 35
           root: 0.08811362304138803


In [324]:
euler_error = calc_euler_error(result.root,par,sim,t=0)
print(f'euler-error = {euler_error:.8f}')

euler-error = 0.00000000


In [325]:
model.simulate()

TypeError: '>' not supported between instances of 'types.SimpleNamespace' and 'int'

In [ ]:
for t in range(5):
    LHS_Euler = sim.C1[t]**(-par.sigma)
    RHS_Euler = (1+sim.rt[t+1])*par.beta * sim.C2[t+1]**(-par.sigma)
    print(f't = {t:2d}: euler-error = {LHS_Euler-RHS_Euler:.8f}')

In [ ]:
#Define some baseline parameters
alpha = 0.3
rho = 0.25
tau = 0.2
n = 0.02
A = 10

#we let $k_t$ be in [0,50]
k_t = np.linspace(0,10, 1000)

#We define the two capital accumulations from our analytical solutions
k_t1_PAYG = ((1-alpha)*(1-tau)*alpha)/((2+rho)*alpha+(1+rho)*(1-alpha)*tau)*A*k_t**alpha
k_t1_FF = ((1-rho*tau-2*tau)/(2+rho))*(1-alpha)*A*k_t**alpha+tau*(1-alpha)*A*k_t**alpha

#We use a root-finder to find the numerical steady state for both pension systems by using the the capital accumulations
obj_ss_PAYG = lambda ss_k_PAYG: ss_k_PAYG - ((1-alpha)*(1-tau)*alpha)/((2+rho)*alpha+(1+rho)*(1-alpha)*tau)*A*ss_k_PAYG**alpha
res_PAYG = optimize.root_scalar(obj_ss_PAYG,bracket=[0.1,100],method='brentq')

obj_ss_FF = lambda ss_k_FF: ss_k_FF - (((1-rho*tau-2*tau)/(2+rho))*(1-alpha)*A*ss_k_FF**alpha+tau*(1-alpha)*A*ss_k_FF**alpha)
res_FF = optimize.root_scalar(obj_ss_FF,bracket=[0.1,100],method='brentq')


#We create plot of two capital accumulations
plt.plot(k_t,k_t1_PAYG, label='PAYG')
plt.plot(k_t,k_t, label='45 degree line')
plt.plot(k_t,k_t1_FF, label='FF')
plt.title('Capital accumulation')
plt.ylabel('k_t+1')
plt.xlabel('k_t')
plt.legend()
plt.grid(True)
plt.show()

print(f'Steady state for capital per capita in the PAYG system = {res_PAYG.root:.3f}')
print(f'Steady state for capital per capita in the FF system = {res_FF.root:.3f}')


We see that there is a larger amount of capital accumulation per capita in the Fully Funded system than in the PAY-AS-YOU-GO system, which results in a higher capital per capita in steady state in the Fully Funded system than in a the PAY-AS-YOU-GO system. This happens because in the Fully Funded system, contributions from the young households are invested in capital in period $t$, and then in period t+1, when the young from period $t$ now are the old households, they receive their contribution with interest. In contrast to the PAY-AS-YOU-GO system, where the contributions from the young households are not involved with capital/investing, but instead are given directly to the old households in the same period.

In [ ]:
#Define the stedy state for rent, wage and savings in both pension systems by using the found steady state for capital per capita
r_ss_PAYG = alpha*A*res_PAYG.root**(alpha-1)
w_ss_PAYG = (1-alpha)*A*res_PAYG.root**alpha
s_ss_PAYG = (1-rho)/(2+rho)*w_ss_PAYG-tau*(1+rho)/((2+rho)*(1+r_ss_PAYG))*w_ss_PAYG

obj_ss_PAYG = lambda c: - (np.log(c[0]) + 1/(1+rho)*np.log(c[1]))

budget_constraint_1 = lambda c_1t: c_1t - ((1-tau)*w_ss_PAYG-s_ss_PAYG)
budget_constraint_2 = lambda c_2t_1: c_2t_1 - ((1+r_ss_PAYG)*s_ss_PAYG+w_ss_PAYG*tau)
euler_constraint = lambda c_1t, c_2t_1: c_1t - (c_2t_1*(1+rho)/r_ss_PAYG+1)
constraint_1 = ({'type': 'eq', 'fun': budget_constraint_1})
constraint_2 = ({'type': 'eq', 'fun': budget_constraint_2})
#constraint_3 = ({'type': 'eq', 'fun': euler_constraint})
bounds = ((1e-8, 10), (1e-8, 10))

c0 = [2,2]

res = optimize.minimize(obj_ss_PAYG, c0, method='SLSQP', bounds=bounds, constraints=[constraint_1, constraint_2])

solve_c1 = res.x[0]
solve_c2 = res.x[1]

print('c1', solve_c1)
print('c2', solve_c2)

def obj_func(c1,c2,rho):
    return np.log(c1) + 1/(1+rho)*np.log(c2)

def obj(c, rho):
    return obj_func(c[0],c[1])

def ineq1(c, rho, tau, w_ss_PAYG, s_ss_PAYG):
    return c[0] - ((1-tau)*w_ss_PAYG-s_ss_PAYG)

def ineq2(c, tau, r_ss_PAYG, w_ss_PAYG, s_ss_PAYG):
    return c[1] - ((1+r_ss_PAYG)*s_ss_PAYG+w_ss_PAYG*tau)

ineq1_con = ({'type': 'ineq', 'fun': ineq1, 'args': (rho, tau, w_ss_PAYG, s_ss_PAYG)})
ineq2_con = ({'type': 'ineq', 'fun': ineq2, 'args': (tau, r_ss_PAYG, w_ss_PAYG, s_ss_PAYG)})
bounds = ((1e-8, 10), (1e-8, 10))
result = optimize.minimize(obj,c0,method='SLSQP',bounds=bounds,constraints=[ineq1_con, ineq2_con])



# Further analysis

**Extension of the OLG model: Population growth**

We now perform an extension to our OLG model, so population is no longer constant, but instead growing. The population grows constant with $n$ for every period, so $L_{t+1}=(1+n)L_t$.

Capital accumulation with population growth is in

PAYG system: $k_{t+1}(1+n)=(1-\delta)k_t+s_t$ 

FF system: $k_{t+1}(1+n)=(1-\delta)k_t+s_t+w_t\tau$

,but we do still assume that the old spend their entrie savings, so $\delta = 1 $.

In [ ]:
#We define the capital accumulation after the extension in both systems, by multiple both with 1/(1+n)
ex_k_t1_PAYG = (1/(1+n))*(((1-alpha)*(1-tau)*alpha)/((2+rho)*alpha+(1+rho)*(1-alpha)*tau)*A*k_t**alpha)
ex_k_t1_FF = (1/(1+n))*(((1-rho*tau-2*tau)/(2+rho))*(1-alpha)*A*k_t**alpha+tau*(1-alpha)*A*k_t**alpha)

#We use a root-finder to find the numerical steady state for both pension systems by using the the capital accumulations
obj_ss_ex_PAYG = lambda ss_k_ex_PAYG: ss_k_ex_PAYG - (1/(1+n))*(((1-alpha)*(1-tau)*alpha)/((2+rho)*alpha+(1+rho)*(1-alpha)*tau)*A*ss_k_ex_PAYG**alpha)
res_ex_PAYG = optimize.root_scalar(obj_ss_ex_PAYG,bracket=[0.1,100],method='brentq')

obj_ss_ex_FF = lambda ss_k_ex_FF: ss_k_ex_FF - ((1/(1+n))*(((1-rho*tau-2*tau)/(2+rho))*(1-alpha)*A*ss_k_ex_FF**alpha+tau*(1-alpha)*A*ss_k_ex_FF**alpha))
res_ex_FF = optimize.root_scalar(obj_ss_ex_FF,bracket=[0.1,100],method='brentq')

plt.plot(k_t,ex_k_t1_PAYG, label='PAYG with pop. growth')
plt.plot(k_t,k_t, label='45 degree line')
plt.plot(k_t,ex_k_t1_FF, label='FF with pop. growth')
plt.plot(k_t,k_t1_PAYG, label='PAYG')
plt.plot(k_t,k_t1_FF, label='FF')
plt.title('Capital accumulation')
plt.ylabel('k_t+1')
plt.xlabel('k_t')
plt.legend()
plt.grid(True)
plt.show()

print(f'Steady state for capital per capita in the PAYG system without population growth = {res_PAYG.root:.3f}')
print(f'Steady state for capital per capita in the PAYG system with population growth = {res_ex_PAYG.root:.3f}')

print(f'Steady state for capital per capita in the FF system without population growth = {res_FF.root:.3f}')
print(f'Steady state for capital per capita in the FF system with population growth = {res_ex_FF.root:.3f}')

We can see that after we extend our OLG model with population growth the accumulation of capital per capita becomes lower in both systems, which results in a lower capital per capita in steady state in both systems. This happens because a bigger population hollows out the total capital, which results in a lower level of capital per capita.

**Capital accumulation and steady state with different parameter-values (focus on tau)**

In [ ]:
#Create a interactive figure 
def plot(alpha=0.3, tau = 0.2, rho = 0.25, A = 10, n=0.02):  
    alpha = alpha
    tau = tau
    rho = rho
    n = n
    k_t = np.linspace(0,50, 1000)
    k_t1_PAYG = ((1-alpha)*(1-tau)*alpha)/((2+rho)*alpha+(1+rho)*(1-alpha)*tau)*A*k_t**alpha
    k_t1_FF = ((1-rho*tau-2*tau)/(2+rho))*(1-alpha)*A*k_t**alpha+tau*(1-alpha)*A*k_t**alpha
    ex_k_t1_PAYG = (1/(1+n))*(((1-alpha)*(1-tau)*alpha)/((2+rho)*alpha+(1+rho)*(1-alpha)*tau)*A*k_t**alpha)
    ex_k_t1_FF = (1/(1+n))*(((1-rho*tau-2*tau)/(2+rho))*(1-alpha)*A*k_t**alpha+tau*(1-alpha)*A*k_t**alpha)
    fig = plt.figure()
    ax1 = fig.add_subplot(1,1,1)
    ax1.plot(k_t,ex_k_t1_PAYG, label='PAYG with pop. growth')
    ax1.plot(k_t,ex_k_t1_FF, label='FF with pop. growth')
    ax1.plot(k_t,k_t, label='45 degree line')
    plt.title('Capital accumulation for different parameter-values')
    ax1.set_xlabel("$k_t$")
    ax1.set_ylabel("$k_{t+1}$")
    ax1.legend()
    ax1.set_xlim(0, 10)
    ax1.set_ylim(0, 10)
    plt.grid(True)
    plt.show()
    
def plot_interact():
    widgets.interact(plot,
                
                 alpha=widgets.FloatSlider(
                     description="alpha", min=0.1, max=1, step=0.05, value=0.3),
                 tau=widgets.FloatSlider(
                     description="tau", min=0.1, max=1, step=0.05, value=0.2),
                 rho=widgets.FloatSlider(
                     description="rho", min=0.1, max=1, step=0.05, value=0.25),
                 A=widgets.FloatSlider(
                     description="A", min=0, max=20, step=1, value=10),
                 n=widgets.FloatSlider(
                     description="n", min=0.001, max=0.5, step=0.005, value=0.02)

                    
    );

plot_interact()

When $A$ increases then accumulation of capital per capita and capital per capita in steady state increases in both systems and vice versa. The intuition behind this is, that a higher technology level increases the marginal product of capital and labor and therefore the rent and wage, which leads to higher savings and contribution, which results in increased accumulation of capital per capita and capital per capita in steady state, ceteris paribus.

When $\rho$ increases then accumulation of capital per capita and capital per capita in steady state decreases in both systems and vice versa. The intuition behind this is, that a higher level of impatience for the young households lead to higher consumption when they are young, and lower consumption, when they are old, ceteris paribus. For that to happen the young households must reduce their savings, which decreases the accumulation of capital per capita and capital per capita in steady state, ceteris paribus.

When $n$ increases then accumulation of capital per capita and capital per capita in steady state decreases in both systems and vice versa. The intuition behind this is, that a higher level population growth lead a higher population level and therefore a greater hollow out of total capital, which results in a lower level of capital per capita. 

When $\tau$ increases then the accumulation of capital per capita and capital per capita in steady state decreases in the PAYG systems and vice versa, but has not effect in the FF system. The intuition behind is that, $\tau$ increases the contribution from the young households, which reduces the saving, which leads to a decreased accumulation of capital per capita and capital per capita in steady state, in the PAYG system, ceteris paribus. In the FF system $\tau$ also increases the contribution from the young households, which reduces the saving ceteris paribus. In contrast to the PAYG the contribution functions just like the the saving (same interest) ceteris paribus, and therefore the level of investing in capital is the same no matter $\tau$. 

The smaller $\alpha$ is the smaller the accumulation of capital per capita and capital per capita in steady state in both systems are ceteris paribus. The intution is a small $\alpha$ means, that the share of capital used in production is small (not important), which results in lowered accumulation of capital per capita and capital per capita in steady state.
When $\alpha$ is around 0.35-0.4 the accumulation of capital per capita and capital per capita in steady state are at their highest level in both systems ceteris paribus. 
If $\alpha$ increases from 0.4 then the accumulation of capital per capita and capital per capita in steady state decrease in both systems and if $\alpha$ is high enough, then there will not even be steady state for capital per capita in both systems ceteris paribus.
The difference between capital per capita in steady state in the two systems also decrease, when $\alpha$ increases ceteris paribus. 

# Conclusion

We can conclude that the accumulation of capital per capita and capital per capita in steady state are higher in **Fully Funded** pension system than in a **Pay-As-You-Go** pension system with constant population and population growth ceteris paribus. 

If there is population growth, then the accumulation of capital per capita and capital per capita in steady state in both systems decrease ceteris paribus. 

The technology level has a positive effect on the accumulation of capital per capita and capital per capita in steady state in both systems with population growth ceteris paribus. The impatience level has a negative effect on the accumulation of capital per capita and capital per capita in steady state in both systems with population growth ceteris paribus.

The contribution share has a negative effect on the accumulation of capital per capita and capital per capita in steady state in the **PAYG** system with population growth, but no effect in **FF** system with population growth ceteris paribus.

The accumulation of capital per capita and capital per capita in steady state in both systems are at the highest level, when the share capital used in production is around 0.35-0.4 ceteris paribus. 